In [1]:
# import common packages
import numpy as np

import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

# lib from Qiskit AQUA Chemistry
from qiskit_aqua_chemistry import FermionicOperator

# lib from optimizer and algorithm
from qiskit_aqua.operator import Operator
from qiskit_aqua import (get_algorithm_instance, get_optimizer_instance, get_variational_form_instance)

# lib for driver
from qiskit_aqua_chemistry.drivers import ConfigurationManager
from collections import OrderedDict

In [2]:
from qiskit_aqua_chemistry.drivers import ConfigurationManager
from collections import OrderedDict
cfg_mgr = ConfigurationManager()
pyscf_cfg = OrderedDict([('atom', 'H .0 .0 .0; H .0 .0 0.735'), ('unit', 'Angstrom'), ('charge', 0), ('spin', 0), \
                         ('basis', 'sto3g')])
section = {}
section['properties'] = pyscf_cfg
driver = cfg_mgr.get_driver_instance('PYSCF')
molecule = driver.run(section)

In [3]:
ferOp = FermionicOperator(h1=molecule._one_body_integrals, h2=molecule._two_body_integrals)
qubitOp_jw = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp_jw.chop(10**-10)

# Using exact eigensolver to get the smallest eigenvalue
exact_eigensolver = get_algorithm_instance('ExactEigensolver')
exact_eigensolver.init_args(qubitOp_jw, k=1)
ret = exact_eigensolver.run()

# print(qubitOp_jw.print_operators())

print('The exact ground state energy is: {}'.format(ret['energy']))
print('The Hartree Fock Electron Energy is: {}'.format(molecule._hf_energy - molecule._nuclear_repulsion_energy))

The exact ground state energy is: -1.8572750302023815
The Hartree Fock Electron Energy is: -1.8369679912029842


In [4]:
# particle hole transformation
newferOp, energy_shift = ferOp.particle_hole_transformation(num_particles=2)
print('Energy shift is: {}'.format(energy_shift))
newqubitOp_jw = newferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
newqubitOp_jw.chop(10**-10)

exact_eigensolver = get_algorithm_instance('ExactEigensolver')
exact_eigensolver.init_args(newqubitOp_jw, k=1)
ret = exact_eigensolver.run()

# print(newqubitOp_jw.print_operators())
print('The exact ground state energy in PH basis is {}'.format(ret['energy']))
print('The exact ground state energy in PH basis is {} (with energy_shift)'.format(ret['energy'] - energy_shift))

Energy shift is: 1.8369679912029846
The exact ground state energy in PH basis is -0.020307038999395333
The exact ground state energy in PH basis is -1.85727503020238 (with energy_shift)


In [5]:
# setup VQE 
# setup optimizer, use L_BFGS_B optimizer for example
lbfgs = get_optimizer_instance('L_BFGS_B')
lbfgs.set_options(maxfun=1000, factr=10, iprint=10)

# setup variation form generator (generate trial circuits for VQE)
var_form = get_variational_form_instance('RY')
var_form.init_args(newqubitOp_jw.num_qubits, 3, entangler_map = {0: [1], 1:[2], 2:[3]})

# setup VQE with operator, variation form, and optimzer
vqe_algorithm = get_algorithm_instance('VQE')
vqe_algorithm.setup_quantum_backend()
vqe_algorithm.init_args(newqubitOp_jw, 'matrix', var_form, lbfgs)
results = vqe_algorithm.run()
print("Minimum value: {}".format(results['eigvals'][0].real))
print("Minimum value: {}".format(results['eigvals'][0].real - energy_shift))
print("Parameters: {}".format(results['opt_params']))

Minimum value: -0.02030703897035787
Minimum value: -1.8572750301733425
Parameters: [ 0.46831958  2.91805342  1.65527567  1.61302219  1.16175063 -3.14159265
 -1.57081478  1.02325207 -1.10249379 -3.14159265 -2.98030417  1.57081197
  2.73252951  3.14159265  1.66611688 -0.55334085]
